# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
#mydb = mongo.get_database("mydb")
mydb = mongo['uk_food']

In [8]:
# confirm that our new database was created
if "uk_food" in mongo.list_database_names():
    print("Database 'mydb' exists.")
else:
    print("Database 'mydb' does not exist.")

Database 'mydb' exists.


In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our new database
collections = mydb.list_collection_names()
for collection in collections:
    print(collection)


establishments


In [18]:
# review the collections in our new database
print("Collections in 'mydb' database:")
for collection in collections:
    print(collection)


Collections in 'mydb' database:
establishments


In [20]:
# review a document in the establishments collection
establishments_collection = db["establishments"]
document = establishments_collection.find_one()

In [21]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [22]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [23]:
# Insert the new restaurant into the collection
inserted_id = establishments_collection.insert_one(new_restaurant).inserted_id

In [24]:
# Check that the new restaurant was inserted
if inserted_id:
    print(f"The new restaurant 'Penang Flavours' was successfully inserted with ID: {inserted_id}")
else:
    print("Failed to insert the new restaurant.")

The new restaurant 'Penang Flavours' was successfully inserted with ID: 65a08f69f5ee841430063603


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [25]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
business_type_info = establishments_collection.find_one(query, projection)
print("Business Type Information:")
print(business_type_info)


Business Type Information:
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [26]:
# Update the new restaurant with the correct BusinessTypeID
new_restaurant_query = {"BusinessName": "Penang Flavours"}
new_business_type_id = business_type_info.get("BusinessTypeID")

if new_business_type_id:
    update_query = {"$set": {"BusinessTypeID": new_business_type_id}}
    establishments_collection.update_one(new_restaurant_query, update_query)
    print("The new restaurant 'Penang Flavours' has been updated with the correct BusinessTypeID.")


The new restaurant 'Penang Flavours' has been updated with the correct BusinessTypeID.


In [27]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments_collection.find_one(new_restaurant_query)
print("Updated Restaurant Information:")
print(updated_restaurant)

Updated Restaurant Information:
{'_id': ObjectId('65a08f69f5ee841430063603'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [28]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_documents_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_documents_count}")


Number of documents with LocalAuthorityName 'Dover': 994


In [29]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many({"LocalAuthorityName": "Dover"})
deleted_count = delete_result.deleted_count
print(f"Number of documents deleted: {deleted_count}")


Number of documents deleted: 994


In [30]:
# Check if any remaining documents include Dover
remaining_dover_documents_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {remaining_dover_documents_count}")


Number of remaining documents with LocalAuthorityName 'Dover': 0


In [31]:
# Check that other documents remain with 'find_one'
example_document = establishments_collection.find_one()
print("Example document from the collection:")
print(example_document)

Example document from the collection:
{'_id': ObjectId('65a086b0a10d37071dd880d9'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [32]:
# Change the data type from String to Decimal for longitude and latitude
establishments_collection.update_many({}, [
    {'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}},
    {'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}
])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [33]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [34]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments_collection.update_many({"RatingValue": {"$in": non_ratings}}, {
    '$set': {"RatingValue": None}
})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [35]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments_collection.find_one()

print("Sample Document after Data Type Conversion:")
print("Longitude:", type(sample_document['geocode']['longitude']))
print("Latitude:", type(sample_document['geocode']['latitude']))
print("RatingValue:", type(sample_document['RatingValue']))


Sample Document after Data Type Conversion:
Longitude: <class 'float'>
Latitude: <class 'float'>
RatingValue: <class 'str'>
